In [ ]:
!pip install Selenium
!pip install moviepy
!pip install selenium
!pip install urllib
!pip install requests

In [1]:
import os
import pandas as pd 
import time
import requests 
import re
import urllib
from tqdm import tqdm
from moviepy.editor import *
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
# we need the proper place where the chrome driver is stored.
# in my case it was in my working directory.
DIR_PATH= os.path.dirname(os.path.realpath('__file__')) + "/chromedriver"

# the abilities of valorant players
abilities= [
    'Sky Smoke',
    'Orbital Strike',
    'Stim Beacon',
    'Incendiary',
    'Paint Shells',
    'Showstopper',
    'Boom Bot',
    'Blast Pack',
    'Spycam',
    'Neural Theft',
    'Trapwire',
    'Cyber Cage',
    'Tailwind',
    'Bladestorm',
    'Cloudburst',
    'Updraft',
    'Dark Cover',
    'From the Shadows',
    'Shrouded Step',
    'Paranoia',
    'Fault line',
    'Rolling Thunder',
    'Aftershock',
    'Flashpoint',
    'Toxic Screen',
    "Viper's Pit",
    'Snake Bite',
    'Poison Cloud',
    'Hot hands',
    'Run it Back',
    'Blaze',
    'Curveball',
    'Healing Orb',
    'Resurrection',
    'Barrier Orb',
    'Slow Orb',
    'Recon Bolt',
    'Recon',
    "Hunter's Fury",
    'Owl Drone',
    'Shock Bolt',
    'Dismiss',
    'Empress',
    'Leer',
    'Devour',
    'Turret',
    'Lockdown',
    'Alarmbot',
    'Nanoswarm',
    'Flash',
    'Trailblazer',
    'Regrowth',
    'Seekers',
    'Guiding Light',
    'Molly'
]
# title will store all the titles for the giuds
title= []

In [3]:
# since the dummy links have abilities specified in them but spaces are replaced with %20
abilities= [s.replace(" ", "%20") for s in abilities]

In [4]:
# setting up the enviroment for out drivers
options= webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
os.environ["webdriver.chrome.driver"]= DIR_PATH

In [5]:
# parent url is the url where these dummy links are, i am calling them dummy links because after clicking those links the video page 
# pop's up 
parent_url = "https://blitz.gg/valorant/guides?agent=all&difficulty=all&map=all&side=both&ability=all"
parent_driver = webdriver.Chrome(chrome_options=options, executable_path= DIR_PATH)   # using the chromedriver stored in DIR_PATH
parent_driver.get(parent_url)    # loading the url in parent_driver

In [6]:
hover_links= []  # this will have all dummy links 
topics_xpath = '//a[@class="LoadMoreButton__LoadMore-sc-1qhz8lq-0 iRYlOI"]' # this is the path for the load more button.
time.sleep(5)  # initial sleep time until the load more button appears in page


In [7]:
# this function will take all the dummy links present on page currently
def get_links():
    # getting all <a> tags
    lnks=parent_driver.find_elements_by_tag_name("a")
    # traverse list
    for lnk in lnks:
    # get_attribute() to get all href
        dummy_link= str(lnk.get_attribute('href'))
        # getting all dummy links we need since they start with 'https://blitz.gg/valorant/maps/'
        if dummy_link.startswith('https://blitz.gg/valorant/maps/'):
            hover_links.append(dummy_link)


In [8]:
# since i dont know how many times we need to click on load more buttom i randomly took 200 as large enough.
for i in range(200):
    try: 
        # gettings dummy links
        get_links()
        time.sleep(2) # my network is slow so i am ading a 2 sec delay
        # going to the bottom of the page since load more is somewhere in bottom
        parent_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # wait until the button is interactable so that we can click it
        WebDriverWait(parent_driver, 20).until(EC.visibility_of_element_located((By.XPATH, topics_xpath)))
        # get the element
        elem= parent_driver.find_element_by_xpath(topics_xpath)
        # since the element is clickable we can click it
        elem.click()
    except:
        # if there is now more "load more" option we will close the window
        parent_driver.close()
        break

In [9]:
len(hover_links)

1200

In [10]:
with open("hover_links.txt", "w") as output:
    output.write(str(hover_links))

In [11]:
# lets take a look how our dummy links look like
hover_links[98] 

'https://blitz.gg/valorant/maps/haven?agent=skye&guide=Grass%20Support%2FInfo%20Flash%20Into%20Garage'

In [12]:
# this fuction will identify which ability is used by its name
def finder (l, file_name):
    for x in l:
        if str(file_name).find(x) !=-1:
            return x.replace("%20", " ")

In [13]:
# the title for video is after 'guide=' 
title= [s.split('guide=',1)[1] for s in hover_links]
# Spaces are represted as %20 in dummy links
title= [s.replace("%20", " ") for s in title]
# abilities_used
ability_used= [finder(abilities, str(s)) for s in hover_links]
# getting the map name
match_map= [re.search('maps/(.+?)?agent', str(s)).group(1)[:-1] for s in hover_links] 
# getting agent name 
agent_name= [re.search('agent=(.+?)&', str(s)).group(1) for s in hover_links]   

In [14]:
v_files= [] # contains the link of the guide

In [15]:
import gc
gc.collect()

200

In [16]:
def get_video(url, i):
    # using chrome driver
    driver.get(url)
    # time.sleep(1.5)
    v_xpath = '//div[@class="Map__VideoFrame-kgp3fv-7 dUFyqf"]'
    WebDriverWait(driver, 4).until(EC.visibility_of_element_located((By.XPATH, v_xpath)))
    time.sleep(0.25)
    # the video link is in <div> tag
    lnks=driver.find_elements_by_tag_name("div")
    # traverse list
    for lnk in lnks:
        # data-div attribute contains the video link we need
        v_file= lnk.get_attribute('data-vid')
        if v_file:
            # driver.close()
            v_files.append(v_file)
            data.append((agent_name[i], ability_used[i], match_map[i], title[i], v_file))
            return
    # closing the page as soon as we are done with it
    

In [17]:
data=[]
driver= webdriver.Chrome(chrome_options=options, executable_path= DIR_PATH)

In [18]:
error= []

In [19]:
# getting all data for every dummy link
i=0
for lnk in tqdm(hover_links):
    try:
        get_video(lnk, i)
        i+=1
    except:
        error.append(i)
        try: 
            get_video(lnk, i)
        except:
            print(i)
        i+=1
driver.close()

100%|██████████| 1200/1200 [1:42:43<00:00,  5.14s/it]


In [20]:
len(data)

1198

In [21]:
import csv
columns= ['agent_name', 'ability_used', 'match_map', 'title', 'link']
with open("data.csv", "w") as f:
    write = csv.writer(f) 
      
    write.writerow(columns) 
    write.writerows(data) 

# Happy valorant guys.      Have fun